In [2]:
!pip install datasets evaluate "transformers[sentencepiece]"

In [3]:
! pip install -U accelerate
! pip install -U transformers

In [4]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

('4.49.0', '1.4.0')

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [6]:
tokenized_datasets['train']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3668
})

In [8]:
tokenized_datasets['validation']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 408
})

In [9]:
tokenized_datasets['test']

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1725
})

In [10]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    report_to="none"
)

In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/var/folders/wp/8mfnwfzn2l5grzntvpnxrl940000gn/T/ipykernel_58391/2700434406.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

Step,Training Loss
500,0.528800
1000,0.293300


TrainOutput(global_step=1377, training_loss=0.3463294556284609, metrics={'train_runtime': 357.0109, 'train_samples_per_second': 30.823, 'train_steps_per_second': 3.857, 'total_flos': 405114969714960.0, 'train_loss': 0.3463294556284609, 'epoch': 3.0})

In [23]:
#调用 Trainer 的 predict 方法，对 tokenized_datasets["validation"]（MRPC 的验证集，包含 408 个样本）进行推理。
#使用训练好的 Trainer对验证集（tokenized_datasets["validation"]）进行预测。
predictions = trainer.predict(tokenized_datasets["validation"])
#返回预测结果（predictions），并打印预测值和真实标签的形状（shape），以便检查输出数据的维度。
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [37]:
#模型输出的预测值（logits），表示每个样本的分类得分，正值越大，表示模型更倾向于该类别。
predictions.predictions[:10]

array([[-3.9880366,  3.415649 ],
       [ 2.5969334, -2.0133705],
       [ 2.0135083, -1.5291071],
       [-3.7645845,  3.0620103],
       [ 2.9759421, -2.266645 ],
       [-3.3909707,  2.6284986],
       [-2.9089735,  2.2585135],
       [-3.9614182,  3.3426323],
       [-3.5333364,  2.7698789],
       [-4.074115 ,  3.4648528]], dtype=float32)

In [33]:
#真实的标签，用于与预测值比较
predictions.label_ids[:408]

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,

In [35]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
print(preds[:408])

[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0
 0 1 1 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 1 1 1
 0 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 1
 1 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0
 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0
 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0
 1]


In [36]:
#计算准确率：
accuracy = (preds == predictions.label_ids).mean()
print(f"Accuracy: {accuracy}")

Accuracy: 0.8651960784313726


In [21]:
! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 12.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.4/22.4 MB 12.3 MB/s eta 0:00:00a 0:00:01


In [38]:
import evaluate
#加载MRPC数据集关联的指标，
metric = evaluate.load("glue", "mrpc")
#评估MRPC数据集在GLUE基准测试中的结果的两个指标
metric.compute(predictions=preds, references=predictions.label_ids)
#模型在验证集上的准确率为 86.51％，F1 分数为 90.50

{'accuracy': 0.8651960784313726, 'f1': 0.9050086355785838}

In [41]:
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate

# 加载数据集
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 分词
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 配置训练参数和模型
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# 定义评估函数
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 初始化 Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 训练模型
trainer.train()
#每轮日志
#Epoch：训练轮数，总共 3 轮，与 TrainingArguments 默认设置一致。
#Training Loss：训练集上的平均损失，反映模型对训练数据的拟合程度。
#Validation Loss：验证集上的平均损失，反映模型的泛化能力。
#Accuracy：验证集上的准确率，由 compute_metrics 计算。
#F1：验证集上的 F1 分数，综合精确率和召回率，适合 MRPC 任务。

/opt/homebrew/anaconda3/envs/AIGC/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/wp/8mfnwfzn2l5grzntvpnxrl940000gn/T/ipykernel_58391/1308564442.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.348576,0.848039,0.894558
2,0.518400,0.520400,0.857843,0.901361
3,0.262900,0.712615,0.852941,0.898305


Using the latest cached version of the module from /Users/imac/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--glue/05234ba7acc44554edcca0978db5fa3bc600eeee66229abe79ff9887eacaf3ed (last modified on Fri Feb 28 14:25:22 2025) since it couldn't be found locally at evaluate-metric--glue, or remotely on the Hugging Face Hub.


TrainOutput(global_step=1377, training_loss=0.3190842561299332, metrics={'train_runtime': 399.3174, 'train_samples_per_second': 27.557, 'train_steps_per_second': 3.448, 'total_flos': 405114969714960.0, 'train_loss': 0.3190842561299332, 'epoch': 3.0})